In [ ]:
# This script performs the analysis of training a convolutional neural network and predicting y given X, where y and X are synthetic benchmark datasets of output and input respectevely, 
# as descirbed in Mamalakis et al. 2022. Here, we also apply many XAI mehtods to explain the predictions of the network. 

 
# citation: 
# Mamalakis, A., E.A. Barnes, I. Ebert-Uphoff (2022) “Investigating the fidelity of explainable 
# artificial intelligence methods for application of convolutional neural networks in geoscience,” 
# arXiv preprint https://arxiv.org/abs/2202.03407. 

# Editor: Dr Antonios Mamalakis (amamalak@colostate.edu)

In [ ]:
#.............................................
# IMPORT STATEMENTS
#.............................................

# local env is AIgeo2

#General Python math functions
import math
#Loading in data (netcdf files)
import h5py
#Handling data
import numpy as np
import netCDF4 as nc
#Plotting figures
import matplotlib.pyplot as plt #Main plotting package

# neural networks package
import keras

#Explaining neural networks using LRP
import innvestigate 


In [ ]:
#.............................................
# LOAD DATA
#.............................................

# load matlab data with the synthetic benchmark
# This data was generated using the matlab script Gen_Synth_SHAPES
filepath = 'synth_data_shapes.mat'
DATA = {}
f = h5py.File(filepath)
for k, v in f.items():
    DATA[k] = np.array(v)

InputX = np.array(DATA['X'])
lats = np.array(DATA['lat'])
lons= np.array(DATA['lon'])
y_synth = np.array(DATA['y'])
Cnt_tr = np.array(DATA['Cnt'])
print('data is loaded') # print message 'data is loaded'

In [ ]:
#.............................................
# DATA MANIPULATION AND SANITY PLOT
#.............................................

Cnt_tr=np.swapaxes(Cnt_tr,-1,1)
InputX=np.swapaxes(InputX,-1,1)

lats=lats.flatten()
lons=lons.flatten()
#Flatten the y time series 
y_synth=y_synth.flatten()

#sanity plot (just for checking I have read the data correclty)
X, Y = np.meshgrid(lons, lats) 
cs = plt.contourf(X, Y, Cnt_tr[9], cmap ="jet")   
cbar = plt.colorbar(cs)   
plt.title('matplotlib.pyplot.contourf() Example') 
plt.show()

In [ ]:
#.............................................
# PREPARE THE DATA FOR TRAINING
#.............................................


# Rename the array to X (inputs) and Y (labels) to stick with machine learning convention
X_all = np.copy(InputX[...,np.newaxis])
Y_all = np.copy(y_synth)

# Change the Y (label) array values to 1 if the sample is above 0 and 0 if the sample is below
Y_all[Y_all > 0] = 1 # square frames cover more area 
Y_all[Y_all <= 0] = 0 # circular frames cover more area

# Convert the Y array into a categorical array. 
Y_all = keras.utils.to_categorical(Y_all)

# Set the fraction of samples that will be used for validation
frac_validate = 0.1

# Separate the X and Y matrices into training and validation sub-sets
# For this problem, we will take the last fraction_validate fraction of samples as our validation dataset
X_train = X_all[:int(-frac_validate*len(X_all))]
Y_train = Y_all[:int(-frac_validate*len(Y_all))]

X_validation = X_all[int(-frac_validate*len(X_all)):]
Y_validation = Y_all[int(-frac_validate*len(Y_all)):]

# Create class weights for training the model. If the dataset is unbalanced, this helps ensure the model
# does not simply start guessing the class that has more samples.
# class_weight = class_weight_creator(Y_train)

# Calculate the number of inputs into the neural network (this will be helpful for the next code cell)
# This value is the number of latitudes times the number of longitudes
number_inputs = X_all.shape[-3:]

In [ ]:
#.............................................
# BUILD THE CONVOLUTIONAL NEURAL NETWORK
#.............................................

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32,(5,5),strides=(2,2),activation='relu',padding='same',input_shape=number_inputs))
#model.add(keras.layers.Conv2D(32,(5,5),strides=(1,1),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(32,(5,5),strides=(1,1),activation='relu',padding='same'))
#model.add(keras.layers.Conv2D(32,(5,5),strides=(1,1),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(64,(3,3),strides=(1,1),activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=128,activation='relu'))
model.add(keras.layers.Dense(units=64,activation='relu'))
model.add(keras.layers.Dense(units=2,activation='softmax'))

#Define the learning rate of the neural network
learning_rate = 0.01

# We will use the stochastic gradient descent (SGD) optimizer, because we have control over
# the learning rate and it is effective for our problem.

model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate),
              loss = 'categorical_crossentropy', 
              metrics=['accuracy'] )

model.summary()    

In [ ]:
#.............................................
# TRAIN THE NEURAL NETWORK
#.............................................

batch_size = 128 #The number of samples the network sees before it backpropagates (batch size)
epochs =  5 #The number of times the network will loop through the entire dataset (epochs)
shuffle = True #Set whether to shuffle the training data so the model doesn't see it sequentially 
verbose = 2 #Set whether the model will output information when trained (0 = no output; 2 = output accuracy every epoch)

###Train the neural network!
model.fit(X_train, Y_train, validation_data=(X_validation, Y_validation), 
          batch_size=batch_size, epochs=epochs, shuffle=shuffle, verbose=verbose) #, class_weight=class_weight)

In [ ]:
#.............................................
# SAVE THE TRAINED NETWORK
#.............................................

model.save('my_model_shapes.h5')

In [ ]:
# load model, including its weights and the optimizer
model = keras.models.load_model('my_model_shapes.h5')
# Show the model architecture
model.summary()
# loss and accuracy in "new model"
loss, acc = model.evaluate(X_validation, Y_validation, verbose=2)
print('Restored model, categorical crossentropy: ', loss)
print('Restored model, categorical accuracy: ', acc)


In [ ]:
#.............................................
# GET EXPLANATIONS FROM MANY XAI METHODS
#.............................................


# Use innvestigate package
# Remove the softmax layer from the model
model_nosoftmax = innvestigate.utils.model_wo_softmax(model)

#Create the "analyzer", or the object that will generate the heatmaps given an input sample
PN_analyzer = innvestigate.create_analyzer('pattern.net', model_nosoftmax)
PN_analyzer.fit(X_train)

PA_analyzer = innvestigate.create_analyzer('pattern.attribution', model_nosoftmax)
PA_analyzer.fit(X_train)

DT_analyzer = innvestigate.create_analyzer('deep_taylor', model_nosoftmax)

lrp_analyzer = innvestigate.create_analyzer('lrp.alpha_beta', model_nosoftmax, alpha=1,beta=0)

lrp2_analyzer = innvestigate.create_analyzer('lrp.epsilon', model_nosoftmax,epsilon=10**(-7))

lrp3_analyzer = innvestigate.create_analyzer('lrp.z', model_nosoftmax)

lrpSeqA_analyzer = innvestigate.create_analyzer('lrp.sequential_preset_a', model_nosoftmax)

lrpSeqAflat_analyzer = innvestigate.create_analyzer('lrp.sequential_preset_a_flat', model_nosoftmax)

gradtinput_analyzer = innvestigate.create_analyzer('input_t_gradient', model_nosoftmax)

intgrad_analyzer = innvestigate.create_analyzer('integrated_gradients', model_nosoftmax)

grad_analyzer = innvestigate.create_analyzer('gradient', model_nosoftmax)

smoothgrad_analyzer = innvestigate.create_analyzer('smoothgrad', model_nosoftmax)


In [ ]:
#.............................................
# XAI LOOP
#.............................................

#First, create some lists to hold the output
PN_heatmaps1_all = []
PA_heatmaps1_all = []
DT_heatmaps1_all = []
LRP_heatmaps1_all = []
LRP2_heatmaps1_all = []
LRP3_heatmaps1_all = []
LRPSEQA_heatmaps1_all = []
LRPSEQAFLAT_heatmaps1_all = []
ItG_heatmaps1_all = []
intG_heatmaps1_all = []
Grad_heatmaps1_all = []
SmooG_heatmaps1_all = []


prediction_all=[]


#We will process all of the testing samples
for sample_ind, sample in enumerate(X_validation):

      # prediction from the model
      sample_prediction = np.argmax(model.predict(sample[np.newaxis,...]))

      PN_heatmap1 = PN_analyzer.analyze(sample[np.newaxis,...])
      PA_heatmap1 = PA_analyzer.analyze(sample[np.newaxis,...])
      DT_heatmap1 = DT_analyzer.analyze(sample[np.newaxis,...]) 
      LRP_heatmap1 = lrp_analyzer.analyze(sample[np.newaxis,...]) 
      LRP2_heatmap1 = lrp2_analyzer.analyze(sample[np.newaxis,...]) 
      LRP3_heatmap1 = lrp3_analyzer.analyze(sample[np.newaxis,...]) 
      LRPSEQA_heatmap1 = lrpSeqA_analyzer.analyze(sample[np.newaxis,...]) 
      LRPSEQAFLAT_heatmap1 = lrpSeqAflat_analyzer.analyze(sample[np.newaxis,...]) 
      ItG_heatmap1 = gradtinput_analyzer.analyze(sample[np.newaxis,...]) 
      intG_heatmap1 = intgrad_analyzer.analyze(sample[np.newaxis,...]) 
      Grad_heatmap1 = grad_analyzer.analyze(sample[np.newaxis,...]) 
      SmooG_heatmap1 = smoothgrad_analyzer.analyze(sample[np.newaxis,...])


      PN_heatmaps1_all.append(PN_heatmap1)
      PA_heatmaps1_all.append(PA_heatmap1)
      DT_heatmaps1_all.append(DT_heatmap1)
      LRP_heatmaps1_all.append(LRP_heatmap1)
      LRP2_heatmaps1_all.append(LRP2_heatmap1)
      LRP3_heatmaps1_all.append(LRP3_heatmap1)
      LRPSEQA_heatmaps1_all.append(LRPSEQA_heatmap1)
      LRPSEQAFLAT_heatmaps1_all.append(LRPSEQAFLAT_heatmap1)
      ItG_heatmaps1_all.append(ItG_heatmap1)
      intG_heatmaps1_all.append(intG_heatmap1)
      Grad_heatmaps1_all.append(Grad_heatmap1)
      SmooG_heatmaps1_all.append(SmooG_heatmap1)

      prediction_all.append(sample_prediction)

        

In [ ]:
#.............................................
# NEXT WE PERFORM DATA MANIPULATION TO GET ALL XAI RESULTS BACK TO THE 'GLOBE' FORMAT
#.............................................

In [ ]:
#Convert the list to an array

PN_heatmaps1_all = np.array(PN_heatmaps1_all)
PA_heatmaps1_all = np.array(PA_heatmaps1_all)
DT_heatmaps1_all = np.array(DT_heatmaps1_all)
LRP_heatmaps1_all = np.array(LRP_heatmaps1_all)
LRP2_heatmaps1_all = np.array(LRP2_heatmaps1_all)
LRP3_heatmaps1_all = np.array(LRP3_heatmaps1_all)
LRPSEQA_heatmaps1_all = np.array(LRPSEQA_heatmaps1_all)
LRPSEQAFLAT_heatmaps1_all = np.array(LRPSEQAFLAT_heatmaps1_all)
ItG_heatmaps1_all = np.array(ItG_heatmaps1_all)
intG_heatmaps1_all = np.array(intG_heatmaps1_all)
Grad_heatmaps1_all = np.array(Grad_heatmaps1_all)
SmooG_heatmaps1_all = np.array(SmooG_heatmaps1_all)



In [ ]:
PN_heatmaps1_all =np.swapaxes(PN_heatmaps1_all ,1,2)
PN_heatmaps1_all =np.swapaxes(PN_heatmaps1_all ,2,3)

PA_heatmaps1_all =np.swapaxes(PA_heatmaps1_all ,1,2)
PA_heatmaps1_all =np.swapaxes(PA_heatmaps1_all ,2,3)

DT_heatmaps1_all =np.swapaxes(DT_heatmaps1_all ,1,2)
DT_heatmaps1_all =np.swapaxes(DT_heatmaps1_all ,2,3)

LRP_heatmaps1_all =np.swapaxes(LRP_heatmaps1_all ,1,2)
LRP_heatmaps1_all =np.swapaxes(LRP_heatmaps1_all ,2,3)

LRP2_heatmaps1_all =np.swapaxes(LRP2_heatmaps1_all ,1,2)
LRP2_heatmaps1_all =np.swapaxes(LRP2_heatmaps1_all ,2,3)

LRP3_heatmaps1_all =np.swapaxes(LRP3_heatmaps1_all ,1,2)
LRP3_heatmaps1_all =np.swapaxes(LRP3_heatmaps1_all ,2,3)

LRPSEQA_heatmaps1_all =np.swapaxes(LRPSEQA_heatmaps1_all ,1,2)
LRPSEQA_heatmaps1_all =np.swapaxes(LRPSEQA_heatmaps1_all ,2,3)

LRPSEQAFLAT_heatmaps1_all =np.swapaxes(LRPSEQAFLAT_heatmaps1_all ,1,2)
LRPSEQAFLAT_heatmaps1_all =np.swapaxes(LRPSEQAFLAT_heatmaps1_all ,2,3)

ItG_heatmaps1_all =np.swapaxes(ItG_heatmaps1_all ,1,2)
ItG_heatmaps1_all =np.swapaxes(ItG_heatmaps1_all ,2,3)

intG_heatmaps1_all =np.swapaxes(intG_heatmaps1_all ,1,2)
intG_heatmaps1_all =np.swapaxes(intG_heatmaps1_all ,2,3)

Grad_heatmaps1_all =np.swapaxes(Grad_heatmaps1_all ,1,2)
Grad_heatmaps1_all =np.swapaxes(Grad_heatmaps1_all ,2,3)

SmooG_heatmaps1_all =np.swapaxes(SmooG_heatmaps1_all ,1,2)
SmooG_heatmaps1_all =np.swapaxes(SmooG_heatmaps1_all ,2,3)



In [ ]:
PN_heatmaps1=np.copy(PN_heatmaps1_all[:,:,:,0,0])
PA_heatmaps1=np.copy(PA_heatmaps1_all[:,:,:,0,0])
DT_heatmaps1=np.copy(DT_heatmaps1_all[:,:,:,0,0])
LRP_heatmaps1=np.copy(LRP_heatmaps1_all[:,:,:,0,0])
LRP2_heatmaps1=np.copy(LRP2_heatmaps1_all[:,:,:,0,0])
LRP3_heatmaps1=np.copy(LRP3_heatmaps1_all[:,:,:,0,0])
LRPSEQA_heatmaps1=np.copy(LRPSEQA_heatmaps1_all[:,:,:,0,0])
LRPSEQAFLAT_heatmaps1=np.copy(LRPSEQAFLAT_heatmaps1_all[:,:,:,0,0])
ItG_heatmaps1=np.copy(ItG_heatmaps1_all[:,:,:,0,0])
intG_heatmaps1=np.copy(intG_heatmaps1_all[:,:,:,0,0])
Grad_heatmaps1=np.copy(Grad_heatmaps1_all[:,:,:,0,0])
SmooG_heatmaps1=np.copy(SmooG_heatmaps1_all[:,:,:,0,0])


In [ ]:
#.............................................
# NEXT WE SAVE ALL XAI RESULTS
#.............................................

In [ ]:
fn = 'PN.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('PN', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(PN_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'PA.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('PA', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(PA_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'DT.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('DT', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(DT_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRP.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRP', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPe.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPe', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP2_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPz.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPz', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP3_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPseqA.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPseqA', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRPSEQA_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPseqAflat.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPseqAflat', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRPSEQAFLAT_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'ItG.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('ItG', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(ItG_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'intG.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('intG', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(intG_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'Grad.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('Grad', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(Grad_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'SmooG.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('SmooG', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(SmooG_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'predictions.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 50000)
times = ds.createVariable('time', 'f4', ('time',))

value2 = ds.createVariable('y_hat_NN', 'f4', ('time',))
value2.units = 'unitless'

value2[:] = np.array(prediction_all)

print('var size ', value2.shape)
ds.close()

In [ ]:
#.............................................
# NEXT WE APPLY XAI METHODS TO EXPLAIN THE 345th TESTING SAMPLE ONLY
#.............................................

In [ ]:
#Remove the softmax layer from the model
model_nosoftmax = innvestigate.utils.model_wo_softmax(model)

#Create the "analyzer" and allow the user to select which class they want to explain
PN_analyzer0 = innvestigate.create_analyzer('pattern.net', model_nosoftmax, neuron_selection_mode="index")
PN_analyzer0.fit(X_train)

PA_analyzer0 = innvestigate.create_analyzer('pattern.attribution', model_nosoftmax, neuron_selection_mode="index")
PA_analyzer0.fit(X_train)

DT_analyzer0 = innvestigate.create_analyzer('deep_taylor', model_nosoftmax, neuron_selection_mode="index")

lrp_analyzer0 = innvestigate.create_analyzer('lrp.alpha_beta', model_nosoftmax, alpha=1,beta=0, neuron_selection_mode="index")

lrp2_analyzer0 = innvestigate.create_analyzer('lrp.epsilon', model_nosoftmax,epsilon=10**(-7), neuron_selection_mode="index")

lrp3_analyzer0 = innvestigate.create_analyzer('lrp.z', model_nosoftmax, neuron_selection_mode="index")

lrpSeqA_analyzer0 = innvestigate.create_analyzer('lrp.sequential_preset_a', model_nosoftmax, neuron_selection_mode="index")

lrpSeqAflat_analyzer0 = innvestigate.create_analyzer('lrp.sequential_preset_a_flat', model_nosoftmax, neuron_selection_mode="index")

gradtinput_analyzer0 = innvestigate.create_analyzer('input_t_gradient', model_nosoftmax, neuron_selection_mode="index")

intgrad_analyzer0 = innvestigate.create_analyzer('integrated_gradients', model_nosoftmax, neuron_selection_mode="index")

grad_analyzer0 = innvestigate.create_analyzer('gradient', model_nosoftmax, neuron_selection_mode="index")

smoothgrad_analyzer0 = innvestigate.create_analyzer('smoothgrad', model_nosoftmax, neuron_selection_mode="index")

In [ ]:
# Explain the 345th testing sample
# and specifically why the class that corresponds to "squares" received very small likelihood
# the "squares" class is 1.
      
PN0_heatmap1 = PN_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
PA0_heatmap1 = PA_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
DT0_heatmap1 = DT_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
LRP0_heatmap1 = lrp_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
LRP20_heatmap1 = lrp2_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
LRP30_heatmap1 = lrp3_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
LRPSEQA0_heatmap1 = lrpSeqA_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
LRPSEQAFLAT0_heatmap1 = lrpSeqAflat_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
ItG0_heatmap1 = gradtinput_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
intG0_heatmap1 = intgrad_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
Grad0_heatmap1 = grad_analyzer0.analyze(X_validation[344][np.newaxis,...],1)
SmooG0_heatmap1 = smoothgrad_analyzer0.analyze(X_validation[344][np.newaxis,...],1)


In [ ]:
#.............................................
# NEXT WE SAVE ALL XAI RESULTS
#.............................................

In [ ]:
PN0_heatmaps1 = np.copy(PN0_heatmap1[:,:,:,0])
PA0_heatmaps1 = np.copy(PA0_heatmap1[:,:,:,0])
DT0_heatmaps1 = np.copy(DT0_heatmap1[:,:,:,0])
LRP0_heatmaps1 = np.copy(LRP0_heatmap1[:,:,:,0])
LRP20_heatmaps1 = np.copy(LRP20_heatmap1[:,:,:,0])
LRP30_heatmaps1 = np.copy(LRP30_heatmap1[:,:,:,0])
LRPSEQA0_heatmaps1 = np.copy(LRPSEQA0_heatmap1[:,:,:,0])
LRPSEQAFLAT0_heatmaps1 = np.copy(LRPSEQAFLAT0_heatmap1[:,:,:,0])
ItG0_heatmaps1 = np.copy(ItG0_heatmap1[:,:,:,0])
intG0_heatmaps1 = np.copy(intG0_heatmap1[:,:,:,0])
Grad0_heatmaps1 = np.copy(Grad0_heatmap1[:,:,:,0])
SmooG0_heatmaps1 = np.copy(SmooG0_heatmap1[:,:,:,0])

In [ ]:
fn = 'PN0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('PN0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(PN0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'PA0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('PA0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(PA0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'DT0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('DT0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(DT0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRP0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRP0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPe0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPe0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP20_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPz0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPz0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRP30_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPseqA0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPseqA0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRPSEQA0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'LRPseqAflat0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('LRPseqAflat0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(LRPSEQAFLAT0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'ItG0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('ItG0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(ItG0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'intG0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('intG0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(intG0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'Grad0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('Grad0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(Grad0_heatmaps1)

print('var size ', value.shape)
ds.close()

In [ ]:
fn = 'SmooG0.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', 1)
lat = ds.createDimension('lat', 65)
lon = ds.createDimension('lon', 65)

times = ds.createVariable('time', 'f4', ('time',))
latss = ds.createVariable('lat', 'f4', ('lat',))
lonss = ds.createVariable('lon', 'f4', ('lon',))
value = ds.createVariable('SmooG0', 'f4', ('time', 'lat', 'lon',))
value.units = 'unitless'

latss[:] = np.copy(lats)
lonss[:] = np.copy(lons)
value[:] = np.copy(SmooG0_heatmaps1)

print('var size ', value.shape)
ds.close()